In [1]:
import pendulum
from datetime import datetime
from mixpanel_jql import JQL, Events, People, raw
from collections import defaultdict
import math
import pandas as pd
from pymongo import MongoClient
import psycopg2
import pytz

In [2]:
client = MongoClient()

db = client.geniePlannerProduction
collection = db.mobile_notifications_notifications

postgres = psycopg2.connect()
pg = postgres.cursor()

In [3]:
def extract_country_data(notification):
    n = notification
    try:
        n['country_code'] = n['country_data'][0]['location']['country_code']
    except:
        try:
            n['country_code'] = n['country_data'][0]['country_code']
        except: pass
    del n['country_data']
    return n
    
n_with_user_data = collection.aggregate([{
    '$lookup': {
        'from': 'users',
        'localField': 'user_id',
        'foreignField': '_id',
        'as': 'country_data'
    }
}])
sent_notifications = list(n_with_user_data)

def company_filter(company_name):
    return company_name.lower() not in ['robscompany', 'geniebelt product team', 'product team', 'qa', 'geniebelt', 'geniebelt aps', 'geniebelt sales team', 'bird', 'alhaprod']

target_users = {
    int(notification['user_id'])
    for notification in sent_notifications
    if company_filter(notification['user_company'])
}

sent_notifications[:] = [extract_country_data(n) for n in sent_notifications if n['user_id'] in target_users]

In [4]:
mixpanel_api_secret = ''

event_selectors = [
    {
        "event": "Progress report generation",
        "selector": 'properties["$device"] == "ios" or properties["$device"] == "android"'
    },
    {
        "event": "Notification opened",
        "selector":'defined(properties["notification_id"]) and properties["notification_id"] != ""'
    }
]

query = JQL(
            mixpanel_api_secret,
            events=Events({
                'event_selectors': event_selectors,
                'from_date': datetime(2018, 3, 15, 15, 0, 0),
                'to_date': datetime.now()
            }),
            people=People(),
            join_params={'type': 'inner'}
        ).map(raw("""
function(row) {
    return {
        'event_name': row.event.name,
        'user_id': row.user.properties.id,
        'event_time': row.event.time,
        'notification_id': row.event.properties.notification_id,
        'country_code': row.event.properties.mp_country_code
    }
}
        """))
full_data = list()
notif_opened_data = {}
progress_reports_data = defaultdict(list)
country_data = {}

def mixpanel_time_to_pendulum(time):
    return pendulum.from_timestamp(time/1000, 'UTC')

for row in query.send():
    full_data.append(row)
    try:
        user_id = int(row['user_id'])
        if not user_id in target_users: continue
        if (row['event_name'] == 'Progress report generation'):
            progress_reports_data[user_id].append(
                mixpanel_time_to_pendulum(row['event_time'])
            )
        if (row['event_name'] == 'Notification opened'):
            notif_opened_data[row['notification_id']] = mixpanel_time_to_pendulum(row['event_time'])
            country_data[row['notification_id']] = row['country_code']
    except:
        continue

In [ ]:
def get_comments_for_package(user_id, package_id):
    query = "SELECT created_at from comments as c inner join comments_packages as cp on cp.comment_id = c.id\
             WHERE c.user_id = %s and cp.package_id = %s"
    pg.execute(query, (user_id, package_id))
    results = pg.fetchall()
    results = list(map(lambda a: pendulum.instance(a[0]), results))
    return results

def get_progressreports_for_package(user_id, package_id):
    query = "SELECT p.created_at from package_progress_reports as p inner join project_members as pm on pm.id = p.project_member_id\
             WHERE pm.user_id = %s and p.package_id = %s"
    pg.execute(query, (user_id, package_id))
    results = pg.fetchall()
    results = list(map(lambda a: pendulum.instance(a[0]), results))
    return results

for notification in sent_notifications:
    progress_reports_data[notification['user_id']] += get_comments_for_package(notification['user_id'], notification['package_id'])
    progress_reports_data[notification['user_id']] += get_progressreports_for_package(notification['user_id'], notification['package_id'])

In [6]:
sorted_progress_reports_data = {
    uid: sorted(pr)
    for (uid, pr) in progress_reports_data.items()
}

In [7]:
def middle_of_list(l):
    return l[math.floor(len(l)/2)]

timezone_data = {key: middle_of_list(pytz.country_timezones(val)) for (key,val) in country_data.items()}

progress_report_map = {}
for notification in sent_notifications:
    user_id = notification['user_id']
    notification_id = str(notification['_id'])
    notification['created_at'] = pendulum.instance(notification['created_at'])
    notification['package_end_date'] = pendulum.instance(notification['package_end_date'])
    if notification['last_progress_report_date']:
        notification['last_progress_report_date'] = pendulum.instance(notification['last_progress_report_date'])
    
    notification_sent_time = notification['created_at']
    notification['timezone'] = 'UTC'
    if 'country_code' in notification.keys():
        notification['timezone'] = middle_of_list(pytz.country_timezones(notification['country_code']))
    if notification_id in notif_opened_data.keys():
        try:
            progress_reports_of_user = sorted_progress_reports_data[user_id]
            for progress_report_time in progress_reports_of_user:
                if progress_report_time < notification_sent_time: continue
                break
            if progress_report_time:
                notification['progress_report_time'] = progress_report_time
        except: pass
        notification['notification_opened_time'] = notif_opened_data[notification_id]
        notification['timezone'] = timezone_data[notification_id]

In [8]:
dataset = []

def timezone_shift(tz):
    dt_base = pendulum.create(2018, 1, 1, tz='UTC')
    dt_shifted = pendulum.create(2018, 1, 1, tz=tz)
    return dt_shifted.diff(dt_base).in_minutes()

for notification in sent_notifications:
    possibly_infinite_attributes = dict()
    if 'notification_opened_time' in notification.keys() and 'progress_report_time' in notification.keys():
        # If notification happens more than half a day before progress report, discard progress report
        if notification['progress_report_time'].diff(notification['notification_opened_time']).in_minutes() > 720:
            notification.pop('progress_report_time', None)

    if 'notification_opened_time' not in notification.keys():
        possibly_infinite_attributes['notification_opened_time'] = math.inf
    else:
        possibly_infinite_attributes['notification_opened_time'] = int(notification['created_at'].diff(notification['notification_opened_time']).in_seconds())
    if 'progress_report_time' not in notification.keys():
        possibly_infinite_attributes['progress_report_time'] = math.inf  
    else:
        possibly_infinite_attributes['progress_report_time'] = int(notification['created_at'].diff(notification['progress_report_time']).in_seconds())
        
    
    local_time_of_day = math.floor(notification['created_at'].seconds_since_midnight()/60)
    local_time_of_day += timezone_shift(notification['timezone'])
    local_time_of_day %= 1440
    
    dataset_notification = {
        'tone': notification['notification_tone'],
        'time_of_day': local_time_of_day,
        'day_of_week': notification['created_at'].day_of_week,
        'who_assigned_role': notification['who_assigned_role'],
        'last_report_delta': notification['created_at'].diff(notification['last_progress_report_date']).in_hours(),
        'last_progress_report_value': notification['last_progress_report_value'],
        'package_duration': notification['package_duration'],
        'end_to_sent_delta': notification['package_end_date'].diff(notification['created_at']).in_days(),
        'sent_to_open_delta': possibly_infinite_attributes['notification_opened_time'],
        'sent_to_reported_delta': possibly_infinite_attributes['progress_report_time'],
    }
    dataset.append(dataset_notification)

In [9]:
DataFrame = pd.DataFrame(dataset)
X = DataFrame.loc[:, DataFrame.columns != 'sent_to_reported_delta']
y = DataFrame['sent_to_reported_delta']

In [10]:
from pandas.api.types import CategoricalDtype
roles = CategoricalDtype(['owner', 'manager'])
tones = CategoricalDtype([1, 2, 3])
days = CategoricalDtype([0, 1, 2, 3, 4, 5, 6])
X.loc[:, 'who_assigned_role'] = X.loc[:, 'who_assigned_role'].astype(roles)
X.loc[:, 'tone'] = X.loc[:, 'tone'].astype(tones)
X.loc[:, 'day_of_week'] = X.loc[:, 'day_of_week'].astype(days)
# 0 is SUNDAY, 1 in MONDAY... 6 is SATURDAY

In [11]:
X_onehot = pd.get_dummies(X)
X_onehot.to_pickle('training_data_X')
y.to_pickle('training_data_y')

In [12]:
list(X_onehot.columns.values)

['end_to_sent_delta',
 'last_progress_report_value',
 'last_report_delta',
 'package_duration',
 'sent_to_open_delta',
 'time_of_day',
 'day_of_week_0',
 'day_of_week_1',
 'day_of_week_2',
 'day_of_week_3',
 'day_of_week_4',
 'day_of_week_5',
 'day_of_week_6',
 'tone_1',
 'tone_2',
 'tone_3',
 'who_assigned_role_owner',
 'who_assigned_role_manager']

In [13]:
DataFrame.loc[DataFrame['sent_to_open_delta'] != math.inf]

,day_of_week,end_to_sent_delta,last_progress_report_value,last_report_delta,package_duration,sent_to_open_delta,sent_to_reported_delta,time_of_day,tone,who_assigned_role
26,4,10,0.99,20,29,35434.0,5.434800e+04,1290,2,manager
28,4,10,0.80,69,7,57911.0,5.793100e+04,1320,2,manager
34,4,6,0.00,357,1,2730.0,inf,480,1,owner
78,5,2,0.80,173,50,864.0,inf,390,1,manager
84,5,23,0.00,2838,1,7.0,inf,990,3,owner
100,5,18,0.70,416,1,4.0,1.400000e+01,1140,2,owner
102,5,5,0.00,5073,7,2143394.0,inf,600,3,manager
107,5,12,0.31,34,27,11678.0,3.939000e+03,690,2,manager
118,5,14,0.15,109,19,14.0,inf,870,1,manager
119,5,0,0.85,37,17,7232.0,7.249000e+03,870,1,manager


In [14]:
len(DataFrame)

2012

In [15]:
len(DataFrame.loc[DataFrame['sent_to_open_delta'] != math.inf])

212

In [16]:
len(DataFrame.loc[DataFrame['sent_to_reported_delta'] != math.inf])

69